# USDA Agriculture Census API

In [1]:
import pandas as pd

%load_ext lab_black

In [2]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

In [3]:
# Docs: https://quickstats.nass.usda.gov/api

In [4]:
# operations =
# url = "https://quickstats.nass.usda.gov/api/api_GET/?key=AC674188-C7A8-3905-9AB7-1FA1DAEDEEF0&commodity_desc=LETTUCE&unit_desc=OPERATIONS&agg_level_desc=COUNTY&year=2017&format=JSON"

In [5]:
# area harvested
url = "https://quickstats.nass.usda.gov/api/api_GET/?key=AC674188-C7A8-3905-9AB7-1FA1DAEDEEF0&commodity_desc=LETTUCE&agg_level_desc=COUNTY&year=2017&format=JSON"

In [6]:
src = pd.read_json(url)

In [7]:
data = []

for r in src["data"]:
    data.append(r)

In [8]:
src = pd.DataFrame(data)

In [9]:
src.short_desc.value_counts()

LETTUCE - ACRES HARVESTED                                          1673
LETTUCE - OPERATIONS WITH AREA HARVESTED                           1634
LETTUCE, FRESH MARKET - ACRES HARVESTED                            1634
LETTUCE, FRESH MARKET - OPERATIONS WITH AREA HARVESTED             1634
LETTUCE, LEAF - ACRES HARVESTED                                    1465
LETTUCE, LEAF - OPERATIONS WITH AREA HARVESTED                     1465
LETTUCE, LEAF, FRESH MARKET - ACRES HARVESTED                      1465
LETTUCE, LEAF, FRESH MARKET - OPERATIONS WITH AREA HARVESTED       1465
LETTUCE, HEAD - ACRES HARVESTED                                     970
LETTUCE, HEAD - OPERATIONS WITH AREA HARVESTED                      970
LETTUCE, HEAD, FRESH MARKET - ACRES HARVESTED                       970
LETTUCE, HEAD, FRESH MARKET - OPERATIONS WITH AREA HARVESTED        970
LETTUCE, ROMAINE - ACRES HARVESTED                                  957
LETTUCE, ROMAINE - OPERATIONS WITH AREA HARVESTED               

In [10]:
src.columns = src.columns.str.lower().str.replace(" ", "_", regex=True)

In [11]:
src.rename(
    columns={"state_ansi": "state_fips", "county_ansi": "county_fips"}, inplace=True
)

In [12]:
src_slim = src[
    [
        "year",
        "agg_level_desc",
        "util_practice_desc",
        "state_name",
        "state_fips",
        "county_name",
        "county_fips",
        "short_desc",
        "domain_desc",
        "value",
    ]
].copy()

### This is a key question: What's the difference between these? 

In [13]:
src_slim.util_practice_desc.value_counts()

ALL UTILIZATION PRACTICES    10091
FRESH MARKET                 10052
Name: util_practice_desc, dtype: int64

In [14]:
src_slim = src_slim[src_slim["util_practice_desc"] == "FRESH MARKET"]

In [15]:
src_slim["agg_level_desc"] = src_slim["agg_level_desc"].str.title()
src_slim["county_name"] = src_slim["county_name"].str.title()
src_slim["state_name"] = src_slim["state_name"].str.title()
src_slim["short_desc"] = src_slim["short_desc"].str.title()
src_slim["domain_desc"] = src_slim["domain_desc"].str.title()

In [16]:
src_slim["value"] = (
    src_slim["value"]
    .str.strip(" ")
    .str.replace("(D)", "0", regex=False)
    .str.replace("(Z)", "0", regex=False)
    .str.replace(",", "", regex=False)
    .astype(int)
)

### What types of reports are available? 

In [17]:
src_slim.short_desc.value_counts()

Lettuce, Fresh Market - Acres Harvested                            1634
Lettuce, Fresh Market - Operations With Area Harvested             1634
Lettuce, Leaf, Fresh Market - Acres Harvested                      1465
Lettuce, Leaf, Fresh Market - Operations With Area Harvested       1465
Lettuce, Head, Fresh Market - Acres Harvested                       970
Lettuce, Head, Fresh Market - Operations With Area Harvested        970
Lettuce, Romaine, Fresh Market - Acres Harvested                    957
Lettuce, Romaine, Fresh Market - Operations With Area Harvested     957
Name: short_desc, dtype: int64

In [18]:
src_slim.sort_values("value", ascending=False)

,year,agg_level_desc,util_practice_desc,state_name,state_fips,county_name,county_fips,short_desc,domain_desc,value
3347,2017,County,FRESH MARKET,California,06,Monterey,053,"Lettuce, Fresh Market - Acres Harvested",Total,169496
8518,2017,County,FRESH MARKET,California,06,Monterey,053,"Lettuce, Head, Fresh Market - Acres Harvested",Total,69017
18226,2017,County,FRESH MARKET,California,06,Monterey,053,"Lettuce, Romaine, Fresh Market - Acres Harvested",Total,60324
13413,2017,County,FRESH MARKET,California,06,Monterey,053,"Lettuce, Leaf, Fresh Market - Acres Harvested",Total,40155
8499,2017,County,FRESH MARKET,Arizona,04,Yuma,027,"Lettuce, Head, Fresh Market - Acres Harvested",Total,38471
...,...,...,...,...,...,...,...,...,...,...
13556,2017,County,FRESH MARKET,Georgia,13,Newton,217,"Lettuce, Leaf, Fresh Market - Acres Harvested",Total,0
4756,2017,County,FRESH MARKET,Virginia,51,Nottoway,135,"Lettuce, Fresh Market - Acres Harvested",Total,0
13555,2017,County,FRESH MARKET,Georgia,13,Morgan,211,"Lettuce, Leaf, Fresh Market - Acres Harvested",Total,0
4755,2017,County,FRESH MARKET,Virginia,51,Lunenburg,111,"Lettuce, Fresh Market - Acres Harvested",Total,0


### Fresh market types

In [19]:
fm_types = [
    "Lettuce, Fresh Market - Acres Harvested",
    "Lettuce, Leaf, Fresh Market - Acres Harvested",
    "Lettuce, Head, Fresh Market - Acres Harvested",
    "Lettuce, Romaine, Fresh Market - Acres Harvested",
]
fm_types

['Lettuce, Fresh Market - Acres Harvested',
 'Lettuce, Leaf, Fresh Market - Acres Harvested',
 'Lettuce, Head, Fresh Market - Acres Harvested',
 'Lettuce, Romaine, Fresh Market - Acres Harvested']

### Limit the dataframe to just the main 'fresh market' types

In [20]:
fm_df = src_slim[
    (src_slim["short_desc"].isin(fm_types)) & (src_slim.domain_desc == "Total")
].copy()

---

### Lettuce, Fresh Market: Acres harvested, by county

In [21]:
acres_df = (
    fm_df[
        (fm_df.short_desc == "Lettuce, Fresh Market - Acres Harvested")
        & (fm_df.domain_desc == "Total")
    ]
    .copy()
    .reset_index(drop=True)
)

In [22]:
acres_df.domain_desc.value_counts()

Total    1634
Name: domain_desc, dtype: int64

In [23]:
acres_df.sort_values("value", ascending=False).head(10)

,year,agg_level_desc,util_practice_desc,state_name,state_fips,county_name,county_fips,short_desc,domain_desc,value
79,2017,County,FRESH MARKET,California,06,Monterey,053,"Lettuce, Fresh Market - Acres Harvested",Total,169496
111,2017,County,FRESH MARKET,California,06,Imperial,025,"Lettuce, Fresh Market - Acres Harvested",Total,35997
117,2017,County,FRESH MARKET,California,06,Santa Barbara,083,"Lettuce, Fresh Market - Acres Harvested",Total,15745
94,2017,County,FRESH MARKET,California,06,Fresno,019,"Lettuce, Fresh Market - Acres Harvested",Total,4702
85,2017,County,FRESH MARKET,California,06,Santa Cruz,087,"Lettuce, Fresh Market - Acres Harvested",Total,4450
114,2017,County,FRESH MARKET,California,06,Riverside,065,"Lettuce, Fresh Market - Acres Harvested",Total,3189
82,2017,County,FRESH MARKET,California,06,San Luis Obispo,079,"Lettuce, Fresh Market - Acres Harvested",Total,2907
81,2017,County,FRESH MARKET,California,06,San Benito,069,"Lettuce, Fresh Market - Acres Harvested",Total,2688
118,2017,County,FRESH MARKET,California,06,Ventura,111,"Lettuce, Fresh Market - Acres Harvested",Total,2257
146,2017,County,FRESH MARKET,Colorado,08,Saguache,109,"Lettuce, Fresh Market - Acres Harvested",Total,1746


---

### Lettuce, Romaine: Acres harvested, by county

In [24]:
romaine_df = (
    fm_df[
        (fm_df.short_desc == "Lettuce, Romaine, Fresh Market - Acres Harvested")
        & (fm_df.domain_desc == "Total")
    ]
    .copy()
    .reset_index(drop=True)
)

In [25]:
romaine_df.domain_desc.value_counts()

Total    957
Name: domain_desc, dtype: int64

In [26]:
romaine_df.sort_values("value", ascending=False).head(10)

,year,agg_level_desc,util_practice_desc,state_name,state_fips,county_name,county_fips,short_desc,domain_desc,value
36,2017,County,FRESH MARKET,California,06,Monterey,053,"Lettuce, Romaine, Fresh Market - Acres Harvested",Total,60324
61,2017,County,FRESH MARKET,California,06,Imperial,025,"Lettuce, Romaine, Fresh Market - Acres Harvested",Total,12783
67,2017,County,FRESH MARKET,California,06,Santa Barbara,083,"Lettuce, Romaine, Fresh Market - Acres Harvested",Total,6523
41,2017,County,FRESH MARKET,California,06,Santa Clara,085,"Lettuce, Romaine, Fresh Market - Acres Harvested",Total,2595
38,2017,County,FRESH MARKET,California,06,San Benito,069,"Lettuce, Romaine, Fresh Market - Acres Harvested",Total,1785
64,2017,County,FRESH MARKET,California,06,Riverside,065,"Lettuce, Romaine, Fresh Market - Acres Harvested",Total,1485
68,2017,County,FRESH MARKET,California,06,Ventura,111,"Lettuce, Romaine, Fresh Market - Acres Harvested",Total,1317
42,2017,County,FRESH MARKET,California,06,Santa Cruz,087,"Lettuce, Romaine, Fresh Market - Acres Harvested",Total,1302
39,2017,County,FRESH MARKET,California,06,San Luis Obispo,079,"Lettuce, Romaine, Fresh Market - Acres Harvested",Total,444
515,2017,County,FRESH MARKET,New Jersey,34,Cumberland,011,"Lettuce, Romaine, Fresh Market - Acres Harvested",Total,365


---

### Grouping by types. Why don't these add up? 

In [27]:
fm_df.groupby(["year", "short_desc"]).agg({"value": sum}).reset_index()

,year,short_desc,value
0,2017,"Lettuce, Fresh Market - Acres Harvested",252455
1,2017,"Lettuce, Head, Fresh Market - Acres Harvested",134636
2,2017,"Lettuce, Leaf, Fresh Market - Acres Harvested",58235
3,2017,"Lettuce, Romaine, Fresh Market - Acres Harvested",89878


In [28]:
fm_df

,year,agg_level_desc,util_practice_desc,state_name,state_fips,county_name,county_fips,short_desc,domain_desc,value
3268,2017,County,FRESH MARKET,Alabama,01,Lauderdale,077,"Lettuce, Fresh Market - Acres Harvested",Total,0
3269,2017,County,FRESH MARKET,Alabama,01,Madison,089,"Lettuce, Fresh Market - Acres Harvested",Total,0
3270,2017,County,FRESH MARKET,Alabama,01,Morgan,103,"Lettuce, Fresh Market - Acres Harvested",Total,0
3271,2017,County,FRESH MARKET,Alabama,01,Blount,009,"Lettuce, Fresh Market - Acres Harvested",Total,1
3272,2017,County,FRESH MARKET,Alabama,01,Calhoun,015,"Lettuce, Fresh Market - Acres Harvested",Total,0
...,...,...,...,...,...,...,...,...,...,...
19142,2017,County,FRESH MARKET,Wisconsin,55,Waukesha,133,"Lettuce, Romaine, Fresh Market - Acres Harvested",Total,0
19143,2017,County,FRESH MARKET,Wyoming,56,Park,029,"Lettuce, Romaine, Fresh Market - Acres Harvested",Total,0
19144,2017,County,FRESH MARKET,Wyoming,56,Sheridan,033,"Lettuce, Romaine, Fresh Market - Acres Harvested",Total,0
19145,2017,County,FRESH MARKET,Wyoming,56,Lincoln,023,"Lettuce, Romaine, Fresh Market - Acres Harvested",Total,0
